# Notes
`ret[dst + "/box/b"] = np.concatenate((v[:4], v))`
> because classes 0 is bg in tensorpack implementation, and in regression it assumes
> N x num_classes x 4, but in detectron2, it's N x num_categories(num_classes-1) x 4 
> But cls_score both have the same dimension(num_classes)


{RoiAlign, NMS, concate, etc} ops does not contain any parameters, so it's not appears in the weights.

In [1]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import torch
# model_weights = torch.load("/notebooks/rtg_models/model_weights/model_anti_lift_pointrend_1012.pth")
model_weights = torch.load("/notebooks/smart_gate/model_weights/container_number_0602.pth")
model = model_weights['model']
print(len(model))

308


In [2]:
def _convert_conv(src, dst):
    model.pop(f'{src}.weight')
    if src + ".norm.weight" in model:
        model.pop(f'{src}.norm.weight')
        model.pop(f'{src}.norm.bias')
        model.pop(f'{src}.norm.running_var')
        model.pop(f'{src}.norm.running_mean')
    if src + ".bias" in model:
        model.pop(f'{src}.bias')
    print(src, "->", dst)

In [3]:
# conv0
print(model['backbone.bottom_up.stem.conv1.weight'].shape) #cout, cin, h, w
_convert_conv('backbone.bottom_up.stem.conv1', 'conv0')

torch.Size([64, 3, 7, 7])
backbone.bottom_up.stem.conv1 -> conv0


In [4]:
# group0:3
backbone_prefix = "backbone.bottom_up."
for grpid in range(4):
    for blkid in range([3, 4, 6, 3][grpid]):
        _convert_conv(backbone_prefix + f"res{grpid + 2}.{blkid}.conv1",
                      f"group{grpid}/block{blkid}/conv1")
        _convert_conv(backbone_prefix + f"res{grpid + 2}.{blkid}.conv2",
                      f"group{grpid}/block{blkid}/conv2")
        _convert_conv(backbone_prefix + f"res{grpid + 2}.{blkid}.conv3",
                      f"group{grpid}/block{blkid}/conv3")
        if blkid == 0:
            _convert_conv(backbone_prefix + f"res{grpid + 2}.{blkid}.shortcut",
                          f"group{grpid}/block{blkid}/convshortcut")

backbone.bottom_up.res2.0.conv1 -> group0/block0/conv1
backbone.bottom_up.res2.0.conv2 -> group0/block0/conv2
backbone.bottom_up.res2.0.conv3 -> group0/block0/conv3
backbone.bottom_up.res2.0.shortcut -> group0/block0/convshortcut
backbone.bottom_up.res2.1.conv1 -> group0/block1/conv1
backbone.bottom_up.res2.1.conv2 -> group0/block1/conv2
backbone.bottom_up.res2.1.conv3 -> group0/block1/conv3
backbone.bottom_up.res2.2.conv1 -> group0/block2/conv1
backbone.bottom_up.res2.2.conv2 -> group0/block2/conv2
backbone.bottom_up.res2.2.conv3 -> group0/block2/conv3
backbone.bottom_up.res3.0.conv1 -> group1/block0/conv1
backbone.bottom_up.res3.0.conv2 -> group1/block0/conv2
backbone.bottom_up.res3.0.conv3 -> group1/block0/conv3
backbone.bottom_up.res3.0.shortcut -> group1/block0/convshortcut
backbone.bottom_up.res3.1.conv1 -> group1/block1/conv1
backbone.bottom_up.res3.1.conv2 -> group1/block1/conv2
backbone.bottom_up.res3.1.conv3 -> group1/block1/conv3
backbone.bottom_up.res3.2.conv1 -> group1/blo

In [5]:
# fpn lateral
for lvl in range(3, 6):
    _convert_conv(f"backbone.fpn_lateral{lvl}", f"fpn/lateral_1x1_c{lvl}")
    _convert_conv(f"backbone.fpn_output{lvl}", f"fpn/posthoc_3x3_p{lvl}")

backbone.fpn_lateral3 -> fpn/lateral_1x1_c3
backbone.fpn_output3 -> fpn/posthoc_3x3_p3
backbone.fpn_lateral4 -> fpn/lateral_1x1_c4
backbone.fpn_output4 -> fpn/posthoc_3x3_p4
backbone.fpn_lateral5 -> fpn/lateral_1x1_c5
backbone.fpn_output5 -> fpn/posthoc_3x3_p5


In [6]:
# non-relevant ones
model.pop('pixel_mean')
model.pop('pixel_std')
for k in list(model.keys()):
    if "cell_anchors" in k:
        model.pop(k)

In [7]:
# remaining keys
for key, item in model.items():
    print(key)

backbone.top_block.p6.weight
backbone.top_block.p6.bias
backbone.top_block.p7.weight
backbone.top_block.p7.bias
head.cls_subnet.0.weight
head.cls_subnet.0.bias
head.cls_subnet.2.weight
head.cls_subnet.2.bias
head.cls_subnet.4.weight
head.cls_subnet.4.bias
head.cls_subnet.6.weight
head.cls_subnet.6.bias
head.bbox_subnet.0.weight
head.bbox_subnet.0.bias
head.bbox_subnet.2.weight
head.bbox_subnet.2.bias
head.bbox_subnet.4.weight
head.bbox_subnet.4.bias
head.bbox_subnet.6.weight
head.bbox_subnet.6.bias
head.cls_score.weight
head.cls_score.bias
head.bbox_pred.weight
head.bbox_pred.bias
